In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

# Import RAG components
from rag_app import EmbeddingService, VectorStore, DocumentProcessor, RAGEngine
from rag_app.vector_store import Document

# For demo purposes
import json
from datetime import datetime
import numpy as np

print("✅ Imports successful!")


In [ ]:
# Configuration Options
# Choose your API provider by uncommenting the appropriate section below

# === OPTION 1: OpenAI (Default) ===
API_KEY = "your-openai-api-key-here"
BASE_URL = None
EMBEDDING_MODEL = "text-embedding-3-small"
CHAT_MODEL = "gpt-4o-mini"
ENDPOINT_TYPE = "OpenAI"

# === OPTION 2: Nutanix Enterprise AI ===
# API_KEY = "your-nutanix-api-key"
# BASE_URL = "https://your-nutanix-endpoint/v1"
# EMBEDDING_MODEL = "your-embedding-model-name"  # e.g., "text-embedding-3-small"
# CHAT_MODEL = "your-chat-model-name"            # e.g., "llama2-7b-chat"
# ENDPOINT_TYPE = "Nutanix Enterprise AI"
# EMBEDDING_DIMENSION = 1536  # Set this to your model's embedding dimension

# === OPTION 3: Azure OpenAI ===
# API_KEY = "your-azure-api-key"
# BASE_URL = "https://your-resource.openai.azure.com/"
# EMBEDDING_MODEL = "your-embedding-deployment-name"
# CHAT_MODEL = "your-chat-deployment-name"
# ENDPOINT_TYPE = "Azure OpenAI"
# API_VERSION = "2023-05-15"

# === OPTION 4: Custom Endpoint ===
# API_KEY = "your-custom-api-key"
# BASE_URL = "https://your-custom-endpoint/v1"
# EMBEDDING_MODEL = "your-embedding-model"
# CHAT_MODEL = "your-chat-model"
# ENDPOINT_TYPE = "Custom"

# RAG configurations (same for all endpoints)
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
MAX_RETRIEVED_DOCS = 5

print("✅ Configuration set!")
print(f"🌐 Endpoint Type: {ENDPOINT_TYPE}")
print(f"📋 Embedding Model: {EMBEDDING_MODEL}")
print(f"🤖 Chat Model: {CHAT_MODEL}")
print(f"🔗 Base URL: {BASE_URL or 'Default (OpenAI)'}")
print(f"📏 Chunk Size: {CHUNK_SIZE}")
print(f"🔄 Chunk Overlap: {CHUNK_OVERLAP}")

if BASE_URL:
    print("⚠️  Make sure your custom endpoint is OpenAI-compatible!")


In [ ]:
# Initialize RAG components
print("🚀 Initializing RAG System...")

# Note: Make sure to set your API_KEY above!
# For demo purposes, we'll check if API_KEY is set
if API_KEY == "your-openai-api-key-here":
    print("⚠️  Please set your OpenAI API key in the cell above!")
    print("   Set API_KEY = 'sk-...' with your actual key")
else:
    try:
        # Initialize embedding service
        embedding_service = EmbeddingService(
            api_key=API_KEY,
            base_url=BASE_URL,
            model_name=EMBEDDING_MODEL
        )
        
        # Get embedding dimension
        embedding_dim = embedding_service.get_embedding_dimension()
        print(f"📐 Embedding dimension: {embedding_dim}")
        
        # Initialize vector store
        vector_store = VectorStore(dimension=embedding_dim, index_type="flat")
        
        # Initialize document processor  
        document_processor = DocumentProcessor(
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP,
            chunking_strategy="recursive"
        )
        
        # Initialize RAG engine
        rag_engine = RAGEngine(
            embedding_service=embedding_service,
            vector_store=vector_store,
            document_processor=document_processor,
            api_key=API_KEY,
            base_url=BASE_URL,
            model_name=CHAT_MODEL,
            temperature=0.7,
            max_retrieved_docs=MAX_RETRIEVED_DOCS
        )
        
        print("✅ RAG system initialized successfully!")
        
        # Add some sample text
        sample_text = """
        Artificial Intelligence (AI) is a branch of computer science that aims to create 
        intelligent machines that can think and act like humans. AI systems can learn, 
        reason, and make decisions. Machine Learning is a subset of AI that enables 
        computers to learn and improve from experience without being explicitly programmed.
        
        Natural Language Processing (NLP) is another important area of AI that focuses on 
        enabling computers to understand, interpret, and generate human language. NLP is 
        used in applications like chatbots, language translation, and text analysis.
        """
        
        # Add the sample text to the knowledge base
        doc_ids = rag_engine.add_text(
            sample_text, 
            source="AI_overview", 
            metadata={"topic": "artificial_intelligence", "type": "educational"}
        )
        
        print(f"📚 Added sample document with {len(doc_ids)} chunks")
        
        # Test the system with a question
        print("\n💬 Testing the RAG system:")
        question = "What is the difference between AI and Machine Learning?"
        print(f"Question: {question}")
        
        response = rag_engine.ask(question)
        print(f"Answer: {response}")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Make sure you have a valid OpenAI API key!")


In [ ]:
# Example configurations for different endpoints
print("🔧 Configuration Examples:\n")

# Example 1: Nutanix Enterprise AI
print("1️⃣ Nutanix Enterprise AI Configuration:")
print("""
# For Nutanix Enterprise AI
rag_engine = RAGEngine.create_for_nutanix(
    api_key="your-nutanix-api-key",
    base_url="https://your-nutanix-cluster/v1",
    embedding_model="text-embedding-3-small",  # Your embedding model
    chat_model="llama2-7b-chat",               # Your chat model  
    embedding_dimension=1536,                  # Model's embedding dimension
    chunk_size=1000,
    temperature=0.7
)
""")

# Example 2: Azure OpenAI
print("2️⃣ Azure OpenAI Configuration:")
print("""
# For Azure OpenAI
embedding_service = EmbeddingService.create_for_azure(
    api_key="your-azure-key",
    base_url="https://your-resource.openai.azure.com/",
    model_name="your-embedding-deployment",
    api_version="2023-05-15"
)

rag_engine = RAGEngine(
    embedding_service=embedding_service,
    vector_store=vector_store,
    document_processor=document_processor,
    api_key="your-azure-key",
    base_url="https://your-resource.openai.azure.com/",
    model_name="your-chat-deployment"
)
""")

# Example 3: Local API (e.g., Ollama)
print("3️⃣ Local API Configuration:")
print("""
# For local APIs like Ollama
embedding_service = EmbeddingService(
    api_key="not-needed",  # Local APIs often don't need keys
    base_url="http://localhost:11434/v1",
    model_name="nomic-embed-text"
)

rag_engine = RAGEngine(
    embedding_service=embedding_service,
    vector_store=vector_store,
    document_processor=document_processor,
    api_key="not-needed",
    base_url="http://localhost:11434/v1",
    model_name="llama2:7b"
)
""")

print("✨ Choose the configuration that matches your setup!")
